In [3]:
import torch, torchvision, PIL, numpy as np
import pathlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm.auto import tqdm
import pandas as pd
from google.colab import drive
import shutil

Get images and data. Relevant files are found in the respository except the images, which are found here: https://drive.google.com/file/d/1-0yYrZAs-BoonZeXHhS5Cb5jWHzlNFvK/view?usp=sharing.

In [4]:
drive.mount('/content/drive')
with open(f'/content/drive/My Drive/CPEN 291 Datasets/Sports_Dataset/annotations_with_bbox.csv', 'r'):
  shutil.copy("/content/drive/My Drive/CPEN 291 Datasets/Sports_Dataset/annotations_with_bbox.csv", "annotations_with_bbox.csv")

dataset = pd.read_csv('annotations_with_bbox.csv')

with open('/content/drive/My Drive/CPEN 291 Datasets/Sports_Dataset/images.tar.gz', 'r'):
  shutil.copy('/content/drive/My Drive/CPEN 291 Datasets/Sports_Dataset/images.tar.gz', 'images.tar.gz')

drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
!tar xfz images.tar.gz

Create the dataset class.

In [ ]:
class DetectionDataset(torch.utils.data.Dataset):
  def __init__(self, df, image_dir, transform=None):
    super().__init__()

    self.df = df
    self.image_ids = self.df['id'].unique()
    self.image_dir = pathlib.Path(image_dir).resolve()
    self.transform = transform
  
  def __getitem__(self, idx):
    image_id = self.image_ids[idx]
    records = self.df[self.df['id'] == image_id]
    im = PIL.Image.open(self.image_dir/image_id).convert('RGB')
    im = torchvision.transforms.ToTensor()(im)

    # bounding box coordinates are stored with width and height while we want an endpoint
    boxes = records[['x', 'y', 'w', 'h']].values
    boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
    boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
    boxes = torch.tensor(boxes, dtype=torch.int64)
    labels = torch.tensor(records['label'])

    # if self.transform:
    #   im = self.transform(im)

    target = {
        'boxes': boxes,
        'labels': labels,
        'image_id': torch.tensor([idx])
    }

    # if self.transform:
    #   im, target = self.transform(im, target)

    return im, target, image_id

  def __len__(self):
    return self.image_ids.shape[0]

In [ ]:
dataset_full = DetectionDataset(dataset, 'images_data')

In [ ]:
dataset_full[0]

In [ ]:
for i in range(9):
  print(dataset.loc[i])

Now to check how it looks as a picture instead of just numbers.

In [ ]:
classes = {
    # "shin": 1,
    # "thigh": 2,
    # "torso": 3,
    # "forearm": 4,
    # "uparm": 5
    1: 'shin',
    2: 'thigh',
    3: 'torso',
    4: 'forearm',
    5: 'uparm'
}

In [ ]:
def show_img(img, title=None):
  if torch.is_tensor(img):
    img = img.cpu().permute(1,2,0)
  plt.imshow(img)
  if title:
    plt.title(title, color='w')
  plt.axis('off')

def show_boxes(img, sample, classes=None):
  show_img(img)
  ax = plt.gca()
  boxes = sample['boxes']
  labels = sample['labels']
  xmin, xmax = ax.get_xlim()
  ymax, ymin = ax.get_ylim()
  isx, isy = xmax-xmin, ymax-ymin
  for i in range(boxes.shape[0]):
    box = boxes[i]
    label = labels[i]
    # x = max(xmin, box[0])
    # y = max(ymin, box[1])
    # w = min(box[2] - x, xmax - x)
    # h = min(box[3] - y, ymax - y)
    # x = xmin + isx * box[0]
    # y = ymin + isy * box[1]
    # w = isx * (box[2] - box[0])
    # h = isy * (box[3] - box[1])
    x = box[0]
    y = box[1]
    w = box[2] - x
    h = box[3] - y
    bbox = patches.Rectangle((x, y), w, h, ec='r', fc='none')
    ax.add_patch(bbox)
    if classes:
      plt.text(x, y, classes[label.item()], backgroundcolor='r', c='w')


In [ ]:
im, result, _ = dataset_full[0]

show_boxes(im, result) #plotting with labels makes it impossible to see
